# Attendance Record

#### Stage 1: Import relevant modules

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt

#### Stage 2: Import relevant attendance data from CWDatabase.db based on input

In [ ]:
# while True:
#     s = input('Please enter a student in the format \'S\'<ID> i.e., S1: '
#         +"\n'q' - quit\n")
#     if s=="q":
#         break
#     elif s=="a":
#         add_an_item()
#     elif s=="d":
#         delete_an_item()# define the function
#     else:
#         print ("sorry no such option")

In [ ]:
################# FUNCTION ###################
def getStudentAtt(id, moduleCode):
    

#### Stage 3: Plot list of attendance and graph based on given student ID.